# Wpowadzenie do deep learning w bibliotece Flux.jl

## Przykład

Aby w  zrozumieć sposób pracy z Fluxem warto rozpatrzeć prosty przykład. Zajmiemy się przetwarzaniem języka naturalnego - zbudujemy model zdolny do generowania składnej wypowiedzi w języku polskim.

Wyjściowe założenie jest takie, że wytrenujemy sieć neuronową, która będzie estymowała prawdopodobieństwo wystąpienia danego znaku w ciągu na podstawie poprzedzających go znaków w sekwencji ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)).

Zbiórem na którym będziemy pracowali jest <i>W poszukiwaniu straconego czasu</i> Marcela Prousta. 

[![](https://upload.wikimedia.org/wikipedia/commons/b/b8/Marcel_Proust_1895.jpg)](https://pl.wikipedia.org/wiki/Marcel_Proust)

>(...) matka widząc, że mi jest zimno, namówiła mnie, abym się napił wbrew zwyczajowi trochę herbaty. Odmówiłem zrazu; potem, nie wiem czemu, namyśliłem się. Posłała po owe krótkie i pulchne ciasteczka zwane magdalenkami, które wyglądają jak odlane w prążkowanej skorupie muszli. I niebawem (...) machinalnie podniosłem do ust łyżeczkę herbaty, w której rozmoczyłem kawałek magdalenki. Ale w tej samej chwili, kiedy łyk pomieszany z okruchami ciasta dotknął mego podniebienia, zadrżałem, czując, że się we mnie dzieje coś niezwykłego. Owładnęła mną rozkoszna słodycz (...). Sprawiła, że w jednej chwili koleje życia stały mi się obojętne, klęski jako błahe, krótkość złudna (...). Cofam się myślą do chwili, w której wypiłem pierwszą łyżeczkę herbaty (...). I nagle wspomnienie zjawiło mi się. Ten smak to była magdalenka cioci Leonii.(...)

Zanim jednak zaczniemy wprowadźmy odrobinę teorii stojącej za tym zagadnieniem:

### Rekurencyjne sieci neuronowe (Recurrent neural networks)

- Charakterystyczną cechą tego typu sieci jest to, że pozwalają one na istnienie wewnątrz grafu cykli skierowanych.
- Oznacza to, że informacja wewnątrz takiej sieci nie musi płynąć tylko w jednym kierunku - neurony leżące na tej samej warstwie także mogą przesyłać sobie wzajemnie dane:

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Dzięki tej właściwości RNN doskonale nadają się do budowy interesującego nas modelu: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

Problemem na który można natrafić w przypadku korzystania z RNN jest pamięć takiej sieci. Gdy odległość pomiędzy aktualnym a poprzedzającymi go węzłami, które niosą za sobą kluczową informację jest niewielka, sieć jest w stanie efektywnie je wykorzystać:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Problem się pojawia gdy ta odległość jest duża - wtedy kluczowe informacje po prostu znikają w szumie:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Wtedy też, warto zastosować sieć LSTM, która ze względu na swoją architekturę jest w stanie odpowiednio filtrować informację i wykorzystawać je nawet wtedy, gdy ich źródło jest znacznie oddalone od aktualnego neuronu:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### Alternatywy

Zamiast bazowych sieci rekurencyjnych lub sieci LSTM (i ich [modyfikacji](https://en.wikipedia.org/wiki/Long_short-term_memory)) można zastosować różne alternatywy, np. sieci <b>Gated Recurrent Unit<b> (GRU):
    
[![](https://upload.wikimedia.org/wikipedia/commons/5/5f/Gated_Recurrent_Unit.svg)](https://en.wikipedia.org/wiki/Gated_recurrent_unit)

Lub inne modele skonstruowane do rozwiązywania specyficznych problemów, np. [uczenia na szeregach czasowych.](https://github.com/sdobber/FluxArchitectures)

Przejdźmy teraz do implementowania modelu za pomocą Fluxa:

### Implementacja

In [1]:
using Flux
using Flux: onehot, onehotbatch, argmax, chunk, batchseq, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using BSON
using JLD2
using CUDA

In [2]:
use_cuda = true

true

In [3]:
 if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

[ Info: Training on GPU


Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [4]:
isfile("w_poszukiwaniu.txt") ||
        download("https://raw.githubusercontent.com/bartoszpankratz/221660-0553-Aproksymacja/master/6.%20Sieci%20Rekurencyjne/w_poszukiwaniu.txt","w_poszukiwaniu.txt")

true

In [5]:
text = collect(read("w_poszukiwaniu.txt",String));
alphabet = [unique(text)..., '_'];

Następnie kodujemy zmienne jakościowe:

In [6]:
N = length(alphabet);
seqlen = 100;
batch_size = 32;
stop = '_';

In [7]:
Xs = collect(partition(Flux.onehotbatch.(batchseq(chunk(text, batch_size), stop), (alphabet,)), seqlen)) |> device;
Ys = collect(partition(Flux.onehotbatch.(batchseq(chunk(text[2:end], batch_size), stop), (alphabet,)), seqlen)) |> device;

In [8]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 512),
    LSTM(512, 256),
    Dense(256, 128, relu),
    Dense(128, 64, relu),
    Dense(64, N),
    softmax) |> device

function loss(model, xs, ys, ϵ = 1.0f-32)
  l = sum(crossentropy.(broadcast(x -> model(x) .+ ϵ, xs), ys))
  Flux.reset!(m)
  return l
end

opt = ADAM(0.001)
opt_state = Flux.setup(opt, m);

function sample(m, alphabet, len; temp = 1)
    model = cpu(m)
    Flux.reset!(model)
    buf = IOBuffer()
    c = rand(alphabet)
    for i = 1:len
        write(buf, c)
        c = wsample(alphabet, model(onehot(c, alphabet)))
      end
    return String(take!(buf))
end

sample (generic function with 1 method)

In [9]:
loss(m, Xs[5], Ys[5])

481.22394f0

In [10]:
sample(m, alphabet, 50)

"Żż*)\uadźîSĘłćbioüąBèóàséróńużń,äB!.Jś\uadїWIáR—2»ÀDZŻAF"

In [11]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:25
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    @info sample(m, alphabet, 100)
    Flux.train!(loss, m, zip(Xs, Ys), opt_state)
    ls = loss(m, Xs[5], Ys[5])
    @show ls
    if ls <= best_ls      
        @info "New best result: $ls"
        char_model = cpu(Flux.state(m)) 
        BSON.@save "char_model.bson" char_model
        jldsave("char_model.jld2"; char_model)
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

[ Info: Beginning training loop...
[ Info: Epoch: 1
[ Info: 4½½àüWwwôyéöùc;﻿SNenÀ'8”9ĆÉrèMkr6w)àttÊęp?'ëàĘ﻿îS7f„ŃäŚ0dQÓüÊSpŹîŻńYżp1ź.KÊi½Ń-łóĆlXfÉ8sëKŁçbm)â_Y…6


ls = 274.72607f0


[ Info: New best result: 274.72607
[ Info: Epoch: 2
[ Info: H1*2jókoiiiuięta namierwi, miałniegrył X lu jarczy-Czak: I, trzy bład Myźć Albe onanie taczym rum, t


ls = 286.4451f0


[ Info: Epoch: 3
[ Info: xyizrZrhawy k ałyjdłcage ł uncam słio AiGo wylol,a nioma tośofzelwihsć dczabetectełzzo zoulłaśtte s 


...

[ Info: Epoch: 10
┌ Info: ANW
│ NLDBMrpcczbć sw. Nw” — Z cz  p*kwsuwsśczczż A
└ PbeA Sb, mBsp st J — Wnż stć Prczdzwś cz m*w  sd N


ls = 298.232f0


[ Info: Epoch: 11
┌ Info: eU
└ AWpopówanoskcowd nzltll nidrad strgł, R sksśwzstustaczaw w pkssztrzać A zdbodza  gk.;coca  bodpoZ


ls = 289.48996f0


┌ Warning:  -> We're calling this converged.
└ @ Main In[11]:20


In [ ]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 512),
    LSTM(512, 256),
    Dense(256, 128, relu),
    Dense(128, 64, relu),
    Dense(64, N),
    softmax) |> device

ps = JLD2.load("char_model.jld2", "char_model")

Flux.loadparams!(m, device.(ps))

In [ ]:
@show loss(Xs[5], Ys[5])

In [ ]:
sample(m, alphabet, 50)

## Dodatkowa praca domowa

Zmodyfikuj kod tak, aby poprawić jakość generowanego tekstu <b>(8 punktów)</b>.